#**Word2Vec 시각화**

##차례
1. PCA 2D
2. t-SNE
  1. 3D
  2. 2D




---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install gensim==4.0.1

     |████████████████████████████████| 23.9MB 167kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from gensim.models import Word2Vec
import pickle

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


**한국어/영어 모델을 선택해 실행합니다.**

In [ ]:
#선택 1: 한국어 모델 불러오기
with open("/content/drive/covid_ko_w2v(tokens).pkl","rb") as fr: 
    model = pickle.load(fr)

In [ ]:
#선택 2: 모델 불러오기
with open("/content/drive/covid_en_w2v(tokens).pkl","rb") as fr: 
    model = pickle.load(fr)

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


##**Visualization setting**

In [ ]:
from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import pandas as pd 
from gensim.models import KeyedVectors
from IPython.display import set_matplotlib_formats
import numpy as np
import matplotlib.font_manager as fm
from collections import Counter
import plotly
import plotly.graph_objs as go

### **키워드 설정**

In [ ]:
def append_list(sim_words, words):
    
    list_of_words = []
    
    for i in range(len(sim_words)):
        
        sim_words_list = list(sim_words[i])
        sim_words_list.append(words)
        sim_words_tuple = tuple(sim_words_list)
        list_of_words.append(sim_words_tuple)
        
    return list_of_words

input_word = '대통령, 시위, 사재기, 마스크, 백신, 격리, 혐오, 지원금, 여행, 실직' #''안에 키워드들을 입력하세요.
user_input = [x.strip() for x in input_word.split(',')]
result_word = []
    
for words in user_input:
    
        sim_words = model.wv.most_similar(words, topn = 10)
        sim_words = append_list(sim_words, words)
            
        result_word.extend(sim_words)
    
similar_word = [word[0] for word in result_word]
similarity = [word[1] for word in result_word] 
similar_word.extend(user_input)
labels = [word[2] for word in result_word]
label_dict = dict([(y,x+1) for x,y in enumerate(set(labels))])
color_map = [label_dict[x] for x in labels]

## **PCA 2D 시각화**

In [ ]:
def display_pca_scatterplot_3D(model, user_input=None, words=None, label=None, color_map=None, topn=10, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.index_to_key), sample)
        else:
            words = [ word for word in model.wv ]

    word_vectors = np.array([model.wv[w] for w in words])
    
    two_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:2]


    data = []
    count = 0
    
    for i in range (len(user_input)):

                trace = go.Scatter(
                    x = two_dim[count:count+topn,0], 
                    y = two_dim[count:count+topn,1],  
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 12,
                    mode = 'markers+text',
                    marker = {
                        'size': 20,
                        'opacity': 0.8,
                        'color': 2
                    }
       
                )
                

            
                data.append(trace)
                count = count+topn

    trace_input = go.Scatter(
                    x = two_dim[count:,0], 
                    y = two_dim[count:,1],  
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 12,
                    mode = 'markers+text',
                    marker = {
                        'size': 20,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )


            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Malgun Gothic",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Malgun Gothic ",
            size = 15),
        autosize = False,
        width = 2000,
        height = 2000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

In [ ]:
display_pca_scatterplot_3D(model, user_input, similar_word, labels, color_map)

##**t-SNE 시각화**

###3D 시각화

In [ ]:
def display_tsne_scatterplot_3D(model, user_input=None, words=None, label=None, color_map=None, perplexity = 0, learning_rate = 0, iteration = 0, topn=10, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.index_to_key), sample)
        else:
            words = [ word for word in model.wv ]

    word_vectors = np.array([model.wv[w] for w in words])

    three_dim = TSNE(n_components = 3, random_state=0, perplexity = perplexity, learning_rate = learning_rate, n_iter = iteration).fit_transform(word_vectors)[:,:3]


    data = []


    count = 0
    for i in range (len(user_input)):

                trace = go.Scatter3d(
                    x = three_dim[count:count+topn,0], 
                    y = three_dim[count:count+topn,1],  
                    z = three_dim[count:count+topn,2],
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 15,
                    mode = 'markers+text',
                    marker = {
                        'size': 5,
                        'opacity': 0.8,
                        'color': 2
                    }
       
                )
                

            
                data.append(trace)
                count = count+topn

    trace_input = go.Scatter3d(
                    x = three_dim[count:,0], 
                    y = three_dim[count:,1],  
                    z = three_dim[count:,2],
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 15,
                    mode = 'markers+text',
                    marker = {
                        'size': 5,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )


            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Malgun Gothic",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Malgun Gothic ",
            size = 15),
        autosize = False,
        width = 1280,
        height = 1280
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

In [ ]:
display_tsne_scatterplot_3D(model, user_input, similar_word, labels, color_map, 20, 500, 10000)

###2D 시각화

In [ ]:
def display_tsne_scatterplot_2D(model, user_input=None, words=None, label=None, color_map=None, perplexity = 0, learning_rate = 0, iteration = 0, topn=10, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.index_to_key), sample)
        else:
            words = [ word for word in model.wv ]

    word_vectors = np.array([model.wv[w] for w in words])
    two_dim = TSNE(n_components = 2, random_state=0, perplexity = perplexity, learning_rate = learning_rate, n_iter = iteration).fit_transform(word_vectors)[:,:2]


    data = []


    count = 0
    for i in range (len(user_input)):

                trace = go.Scatter(
                    x = two_dim[count:count+topn,0], 
                    y = two_dim[count:count+topn,1],  
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 15,
                    mode = 'markers+text',
                    marker = {
                        'size': 15,
                        'opacity': 0.8,
                        'color': 2
                    }
       
                )
                

            
                data.append(trace)
                count = count+topn

    trace_input = go.Scatter(
                    x = two_dim[count:,0], 
                    y = two_dim[count:,1],  
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 15,
                    mode = 'markers+text',
                    marker = {
                        'size': 15,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )

            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Malgun Gothic",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Malgun Gothic ",
            size = 15),
        autosize = False,
        width = 1280,
        height = 1280
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

In [ ]:
display_tsne_scatterplot_2D(model, user_input, similar_word, labels, color_map, 20, 500, 10000)